In [1]:
import pandas as pd 
import numpy as np 
import os 
import sys

In [2]:
oldfile = '/home/poojaparameswaran/Documents/SoderlingLab/SpatialProteomics/tables/original_results/Vps35-TMT-Module-Results_log2relint.xlsx'
newfile = f'/home/poojaparameswaran/Documents/SoderlingLab/SpatialProteomics/tables/Vps35-TMT-Module-Results.xlsx'
olddata = pd.ExcelFile(oldfile)
newdata = pd.ExcelFile(newfile)



In [3]:
## first compare items in analagous clusters.
op = olddata.parse(0)
nep = newdata.parse(0)
oldgrouped = op.groupby('Membership')
maps = {o: -1 for o in nep['Membership'].unique()}
newgrouped = nep.groupby('Membership')
noverlap = {o: -1 for o in nep['Membership'].unique()}
for newn, newg in newgrouped:
    mx = 0 # max
    oldn = 0
    while oldn < op['Membership'].max()+1:
        overlap = len(set(newg['UniProt']).intersection(set(oldgrouped.get_group(oldn)['UniProt'])))
        if overlap > mx:
            mx = overlap
            if maps[newn] == -1:
                maps[newn] = oldn
                noverlap[newn] = overlap
            else:
                print(f'New group {newn} has more overlap with old group {oldn} than what was previously populated'
                      f'{maps[newn]} with an overlap amt of {overlap}')
                maps[newn] = oldn
                noverlap[newn] = overlap


        oldn +=1
    

New group 4 has more overlap with old group 5 than what was previously populated1 with an overlap amt of 67
New group 5 has more overlap with old group 2 than what was previously populated1 with an overlap amt of 64
New group 6 has more overlap with old group 6 than what was previously populated5 with an overlap amt of 58
New group 8 has more overlap with old group 9 than what was previously populated4 with an overlap amt of 46
New group 9 has more overlap with old group 11 than what was previously populated2 with an overlap amt of 37
New group 10 has more overlap with old group 8 than what was previously populated1 with an overlap amt of 44
New group 12 has more overlap with old group 12 than what was previously populated8 with an overlap amt of 38
New group 16 has more overlap with old group 6 than what was previously populated1 with an overlap amt of 5


In [4]:
22 in noverlap

False

In [5]:
nep = nep.rename(columns={'Membership': 'program_assigned_membership'})
nep.insert(nep.columns.get_loc('Symbol')+1, 'Membership', nep['program_assigned_membership'].map(maps))
nep

,UniProt,Entrez,Symbol,Membership,program_assigned_membership
0,B1AZP2,228836.0,Dlgap4,0,0
1,E9Q1U1,320226.0,Ccdc171,0,0
2,P12382,18641.0,Pfkl,0,0
3,P16330,12799.0,Cnp,0,0
4,P17427,11772.0,Ap2a2,0,0
...,...,...,...,...,...
861,P70663,13602.0,Sparcl1,15,18
862,Q11011,19155.0,Npepps,15,18
863,Q8CGF6,99512.0,Wdr47,15,18
864,Q9CY64,109778.0,Blvra,15,18


In [6]:
import re
re.search(r'\d+', 'M2')

<re.Match object; span=(1, 2), match='2'>

In [7]:
nmod = newdata.parse(1)
nmod.insert(nmod.columns.get_loc('Module')+1, 'OldModuleNumber', nmod['Module'].apply(
    lambda x: maps[int(re.search(r'(\d+)', x).group(1))]
))
nmod

,Module,OldModuleNumber,nProts,Contrast,log2FC,percentControl,SE,Tstatistic,Pvalue,FDR,Padjust,DF,S2,candidate
0,M1,1,114,tg-wt,-0.231464,85.176999,0.011826,-19.573013,4.773948e-82,9.070501e-81,9.070501e-81,4661.000001,0.167397,True
1,M4,5,68,tg-wt,-0.175669,88.535698,0.019455,-9.029710,3.142145e-19,2.985038e-18,5.970075e-18,2775.000000,0.270234,True
2,M3,4,70,tg-wt,-0.099984,93.304322,0.015481,-6.458360,1.239541e-10,7.850424e-10,2.355127e-09,2856.999987,0.176159,True
3,M10,8,46,tg-wt,-0.181463,88.180823,0.029314,-6.190330,7.323388e-10,3.478609e-09,1.391444e-08,1918.000000,0.415046,True
4,M5,2,67,tg-wt,-0.131187,91.307995,0.022558,-5.815519,6.746190e-09,2.563552e-08,1.281776e-07,2734.000000,0.357988,True
5,M15,14,18,tg-wt,0.191043,114.158864,0.034682,5.508355,5.033593e-08,1.593971e-07,9.563827e-07,725.000001,0.227342,True
6,M6,6,60,tg-wt,-0.091449,93.857945,0.017286,-5.290329,1.326683e-07,3.600997e-07,2.520698e-06,2506.000000,0.188250,True
7,M13,13,32,tg-wt,-0.181487,88.179390,0.034473,-5.264662,1.641887e-07,3.899482e-07,3.119586e-06,1299.000000,0.399289,True
8,M12,12,39,tg-wt,-0.095005,93.626919,0.021792,-4.359589,1.386880e-05,2.927858e-05,2.635072e-04,1586.000000,0.194470,True
9,M8,9,49,tg-wt,-0.064253,95.644054,0.017382,-3.696595,2.242644e-04,4.261023e-04,4.261023e-03,2044.000000,0.155441,True


In [8]:
nmod.insert(nmod.columns.get_loc('OldModuleNumber')+1, 'Overlap', nmod.apply(
    lambda x: noverlap[int(re.search(r'(\d+)', x['Module']).group(1))] / x['nProts'], axis=1
))

In [9]:
nmod

,Module,OldModuleNumber,Overlap,nProts,Contrast,log2FC,percentControl,SE,Tstatistic,Pvalue,FDR,Padjust,DF,S2,candidate
0,M1,1,0.991228,114,tg-wt,-0.231464,85.176999,0.011826,-19.573013,4.773948e-82,9.070501e-81,9.070501e-81,4661.000001,0.167397,True
1,M4,5,0.985294,68,tg-wt,-0.175669,88.535698,0.019455,-9.029710,3.142145e-19,2.985038e-18,5.970075e-18,2775.000000,0.270234,True
2,M3,4,0.985714,70,tg-wt,-0.099984,93.304322,0.015481,-6.458360,1.239541e-10,7.850424e-10,2.355127e-09,2856.999987,0.176159,True
3,M10,8,0.956522,46,tg-wt,-0.181463,88.180823,0.029314,-6.190330,7.323388e-10,3.478609e-09,1.391444e-08,1918.000000,0.415046,True
4,M5,2,0.955224,67,tg-wt,-0.131187,91.307995,0.022558,-5.815519,6.746190e-09,2.563552e-08,1.281776e-07,2734.000000,0.357988,True
5,M15,14,1.000000,18,tg-wt,0.191043,114.158864,0.034682,5.508355,5.033593e-08,1.593971e-07,9.563827e-07,725.000001,0.227342,True
6,M6,6,0.966667,60,tg-wt,-0.091449,93.857945,0.017286,-5.290329,1.326683e-07,3.600997e-07,2.520698e-06,2506.000000,0.188250,True
7,M13,13,1.000000,32,tg-wt,-0.181487,88.179390,0.034473,-5.264662,1.641887e-07,3.899482e-07,3.119586e-06,1299.000000,0.399289,True
8,M12,12,0.974359,39,tg-wt,-0.095005,93.626919,0.021792,-4.359589,1.386880e-05,2.927858e-05,2.635072e-04,1586.000000,0.194470,True
9,M8,9,0.938776,49,tg-wt,-0.064253,95.644054,0.017382,-3.696595,2.242644e-04,4.261023e-04,4.261023e-03,2044.000000,0.155441,True


In [10]:
from pathlib import Path
with pd.ExcelWriter(Path(newfile).parent / ('Final_' +Path(newfile).name) ,mode='w') as writer:
    nep.to_excel(writer, sheet_name='Partition', index=False)
    nmod.to_excel(writer, sheet_name='Module Results', index=False)
    op.to_excel(writer, sheet_name='Old Partitions', index=False)
    olddata.parse(1).to_excel(writer, 'Old Module Results', index=False)


In [11]:
Path(newfile).name

'Vps35-TMT-Module-Results.xlsx'